In [1]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
from openai import OpenAI

In [302]:
import os
import pandas as pd
files = os.listdir()
# for i in files:
#     if i.endswith('.xlsx'):
#         print(i)
import json
filepath = 'Flow chart english.xlsx'
os.path.exists(filepath)

# Load Excel (adjust path and sheet name as needed)
df = pd.read_excel(filepath, header=9)
columns_to_drop = ['Process Symbols','Improve content', 'Inspection Problem Description','Improvement goals.1', 
                   'Current Level', 'Improvement goals.2', 'Unnamed: 25', 'Unnamed: 26', 'Expected', 'actual', 'BU', 'Manufacturing CoE-IE',
                   'Number of operators', 'Average working hours at workstations (Sec.)','Unnamed: 34','Unnamed: 35',
                   'Assignment content.1','Single time','Number of jobs','Total working hours', ' Saturation']
df = df.drop(columns_to_drop, axis=1)

print(len(df))
# df.head()
# print(df.to_string(index=False, max_cols=None))

33


In [303]:
df = df.rename(columns={'name': 'Product Number', 
                        'Serial number': 'Workstation number', 
                        'name.1': 'Workstation name',
                        'Serial number.1': 'Serial number',
                        'Unnamed: 10': 'Number of operators',
                        'Unnamed: 11': 'Average Working hours (Sec.)',
                        'Unnamed: 12': 'Current manual labor saturation',
                        'Process': 'Value Judgment Process',
                        'unit': 'Value Judgment unit',
                        })
print(df.to_string(index=False, max_cols=None))

                                      Product Number  Workstation number                                      Workstation name  Serial number                                                                                                                                                                            Assignment content  Manual (Sec.)  Machine (Sec.)  Number of products  Working hours (Sec.)  Number of operators  Average Working hours (Sec.)  Current manual labor saturation Value Judgment Process Value Judgment unit Problem Description Improvement goals ECRS Action Item Classification Current Action Level
                                                 NaN                 8.0                                              0/S Test            1.0                                                                                                                        Take 1 PCS product in each hand and put it on the test machine support           2.01             NaN               

In [304]:
df['Workstation number'] = df['Workstation number'].fillna(method='ffill')
df['Workstation name'] = df['Workstation name'].fillna(method='ffill')
df['Machine (Sec.)'] = df['Machine (Sec.)'].fillna(0)
df['Number of products'] = df['Number of products'].fillna(method='ffill')
df['Working hours (Sec.)'] = df['Working hours (Sec.)'].fillna(method='ffill')
df['Number of operators'] = df['Number of operators'].fillna(method='ffill')
df['Average Working hours (Sec.)'] = df['Average Working hours (Sec.)'].fillna(method='ffill')
df['Current manual labor saturation'] = df['Current manual labor saturation'].fillna(method='ffill')
df['Value Judgment Process'] = df['Value Judgment Process'].fillna(method='ffill')
df['ECRS'] = df['ECRS'].fillna(method='ffill')

C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_19464\3090147063.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Workstation number'] = df['Workstation number'].fillna(method='ffill')
C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_19464\3090147063.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Workstation name'] = df['Workstation name'].fillna(method='ffill')
C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_19464\3090147063.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Number of products'] = df['Number of products'].fillna(method='ffill')
C:\Users\Muhammad Al-Amin\AppData\Local\Temp\ipykernel_19464\3090147063.py:5: FutureWarning: Series.fillna with 'method' is deprecated and w

In [305]:
product_number = '3S6A013-00518-7F'
edition = 'B'
updated  = 263822
approved_by = 'approved by'
reviewer = 'reviewed by'
prepared_by = 'prepared by'
responsible_person = 'responsible person'

# Apply header metadata to all rows
df["Product Number"] = product_number
df["Edition"] = edition
df["Updated"] = updated
df["Approved by"] = approved_by
df["Reviewer"] = reviewer
df["Prepared by"] = prepared_by
df["Responsible Person"] = responsible_person

In [306]:
df.dropna(subset=['Serial number'], inplace=True)
print(df.to_string(index=False, max_cols=None))

  Product Number  Workstation number                                      Workstation name  Serial number                                                                                                                                                                            Assignment content  Manual (Sec.)  Machine (Sec.)  Number of products  Working hours (Sec.)  Number of operators  Average Working hours (Sec.)  Current manual labor saturation Value Judgment Process Value Judgment unit Problem Description Improvement goals ECRS Action Item Classification Current Action Level Edition  Updated Approved by    Reviewer Prepared by Responsible Person
3S6A013-00518-7F                 8.0                                              0/S Test            1.0                                                                                                                        Take 1 PCS product in each hand and put it on the test machine support           2.01            0.00               

In [307]:
df['ECRS'] = df['ECRS'].fillna('N/A')

In [308]:
df.columns
mapping_action_item_classification = {'→': 'Transport', '○': 'Operation', '□': 'Inspect'}
mapping_current_action_level = {'M1': 'Manual', 'M2': 'Semi automation', 'M3': 'Automation', 'M4': 'Advanced automation'}
df['Action Item Classification'] = df['Action Item Classification'].replace(mapping_action_item_classification)
df['Current Action Level'] = df['Current Action Level'].replace(mapping_current_action_level)

In [309]:
df['Workstation number'] = df['Workstation number'].astype('Int64')
df['Serial number'] = df['Serial number'].astype('Int64')
df['Number of products'] = df['Number of products'].astype('Int64')
df['Number of operators'] = df['Number of operators'].astype('Int64')

In [310]:
print(df.head(len(df)).to_string(index=False, max_cols=None))

  Product Number  Workstation number                                      Workstation name  Serial number                                                                                                                                                                            Assignment content  Manual (Sec.)  Machine (Sec.)  Number of products  Working hours (Sec.)  Number of operators  Average Working hours (Sec.)  Current manual labor saturation Value Judgment Process Value Judgment unit Problem Description Improvement goals ECRS Action Item Classification Current Action Level Edition  Updated Approved by    Reviewer Prepared by Responsible Person
3S6A013-00518-7F                   8                                              0/S Test              1                                                                                                                        Take 1 PCS product in each hand and put it on the test machine support           2.01            0.00               

In [55]:
# # Initialize list for documents
# documents = []

# # Group by Segment
# for segment in df["Workstation number"].unique():
#     segment_df = df[df["Workstation number"] == segment].copy()
    
#     # Extract process-level data
#     workstation_number = segment_df["Workstation number"].iloc[0]
#     workstation_name = segment_df["Workstation name"].iloc[0]
#     number_of_products = segment_df["Number of products"].iloc[0]
#     working_hours_sec = segment_df["Working hours (Sec.)"].iloc[0]
#     number_of_operators = segment_df["Number of operators"].iloc[0]
#     average_working_hours_sec = segment_df["Average Working hours (Sec.)"].iloc[0]
#     current_manual_labor_saturation = segment_df["Current manual labor saturation"].iloc[0]
#     value_judgment_process = segment_df["Value Judgment Process"].iloc[0]

#     # Extract operations
#     operations = []
#     for _, row in segment_df.iterrows():
#         operations.append({
#             "serial_number": row["Serial number"],
#             "description": row["Assignment content"],
#             "manual_sec": row["Manual (Sec.)"],
#             "machine_sec": row["Machine (Sec.)"],
#             "value_judgment_unit": row["Value Judgment unit"],
#             "action_item_classification": row["Action Item Classification"],
#             "current_action_level": row["Current Action Level"]
#         })
#         print(row["Current Action Level"])

In [311]:
df.columns

Index(['Product Number', 'Workstation number', 'Workstation name',
       'Serial number', 'Assignment content', 'Manual (Sec.)',
       'Machine (Sec.)', 'Number of products', 'Working hours (Sec.)',
       'Number of operators', 'Average Working hours (Sec.)',
       'Current manual labor saturation', 'Value Judgment Process',
       'Value Judgment unit', 'Problem Description', 'Improvement goals',
       'ECRS', 'Action Item Classification', 'Current Action Level', 'Edition',
       'Updated', 'Approved by', 'Reviewer', 'Prepared by',
       'Responsible Person'],
      dtype='object')

In [337]:
texts = []
doc_ids = []
metadatas = []
for i, doc in enumerate(documents):
    # Workstation document
    workstation_text = "; ".join([
        f"Workstation Number: {doc['Workstation number']}",
        f"Workstation Name: {doc['Workstation name']}",
        f"Number of Products: {doc['Number of products']}",
        f"Working hours (Sec.): {doc['Working hours (Sec.)']}",
        f"Number of Operators: {doc['Number of operators']}",
        f"Cycle Time (Sec.): {doc['Cycle Time (Sec.)']}",
        f"Job saturation (%): {doc['Job saturation (%)']}",
        f"Value Judgment Process: {doc['Value Judgment Process']}",
        f"Improvement Goals: {doc['improvement items']['goals'] if doc['improvement items']['goals'] != '/' else 'No improvements listed'}"
    ])
    texts.append(workstation_text)
    doc_ids.append(f"ws_{i}")
    metadatas.append({
        "product_number": doc["product_number"],
        "workstation_number": str(doc["Workstation number"]),
        "workstation_name": doc["Workstation name"],
        "type": "workstation"
    })
    
    # Operation documents
    for j, op in enumerate(doc["operations"]):
        op_text = "; ".join([
            f"Workstation Number: {doc['Workstation number']}",
            f"Workstation Name: {doc['Workstation name']}",
            f"Serial Number: {op['serial_number']}",
            f"Description: {op['description']}",
            f"Manual (sec): {op['manual_sec']}sec",
            f"Machine (sec): {op['machine_sec']}sec",
            f"Value Judgment Unit: {op['value_judgment_unit']}",
            f"Action Item Classification: {op['action_item_classification']}",
            f"Current Action Level: {op['current_action_level']}"
        ])
        texts.append(op_text)
        doc_ids.append(f"op_{i}_{j}")
        metadatas.append({
            "product_number": doc["product_number"],
            "workstation_number": str(doc["Workstation number"]),
            "workstation_name": doc["Workstation name"],
            "serial_number": str(op["serial_number"]),
            "type": "operation"
        })

In [338]:
print(len(texts))
print(len(doc_ids))
print(len(metadatas))
doc_ids
texts[1]

39
39
39


'Workstation Number: 8; Workstation Name: 0/S Test; Serial Number: 1; Description: Take 1 PCS product in each hand and put it on the test machine support; Manual (sec): 2.01sec; Machine (sec): 0.0sec; Value Judgment Unit: NVA; Action Item Classification: Transport; Current Action Level: Automation'

In [312]:
# Initialize list for documents
documents = []

# Group by Segment
for segment in df["Workstation number"].unique():
    segment_df = df[df["Workstation number"] == segment].copy()
    
    # Extract workstation level data
    workstation_number = segment_df["Workstation number"].iloc[0]
    workstation_name = segment_df["Workstation name"].iloc[0]
    number_of_products = segment_df["Number of products"].iloc[0]
    working_hours_sec = segment_df["Working hours (Sec.)"].iloc[0]
    number_of_operators = segment_df["Number of operators"].iloc[0]
    average_working_hours_sec = segment_df["Average Working hours (Sec.)"].iloc[0]
    current_manual_labor_saturation = segment_df["Current manual labor saturation"].iloc[0]
    value_judgment_process = segment_df["Value Judgment Process"].iloc[0]

    # Extract operation level data
    operations = []
    for _, row in segment_df.iterrows():
        operations.append({
            "serial_number": row["Serial number"],
            "description": row["Assignment content"],
            "manual_sec": row["Manual (Sec.)"],
            "machine_sec": row["Machine (Sec.)"],
            "value_judgment_unit": row["Value Judgment unit"],
            "action_item_classification": row["Action Item Classification"],
            "current_action_level": row["Current Action Level"]
        })
        
    # Extract improvements
    improvements = {
        "problem_description": segment_df["Problem Description"].iloc[0],
        "goals": segment_df["Improvement goals"].iloc[0],
        "ecrs": segment_df["ECRS"].iloc[0],
    }
    
    # Metadata
    metadata = {
        "edition": segment_df["Edition"].iloc[0],
        "updated": segment_df["Updated"].iloc[0],
        "approved_by": segment_df["Approved by"].iloc[0],
        "reviewer": segment_df["Reviewer"].iloc[0],
        "prepared_by": segment_df["Prepared by"].iloc[0],
        "responsible_person": segment_df["Responsible Person"].iloc[0]
    }


# Create document
    document = {
        "product_number": segment_df["Product Number"].iloc[0],
        "Workstation number": workstation_number,
        "Workstation name": workstation_name,
        "Number of products": number_of_products,
        "Working hours (Sec.)": working_hours_sec,
        "Number of operators": number_of_operators,
        "Cycle Time (Sec.)": average_working_hours_sec,
        "Job saturation (%)": current_manual_labor_saturation,
        "Value Judgment Process": value_judgment_process,
        "operations": operations,
        "improvement items": improvements,
        "metadata": metadata
    }
    documents.append(document)        

In [313]:
documents[0]

{'product_number': '3S6A013-00518-7F',
 'Workstation number': 8,
 'Workstation name': '0/S Test',
 'Number of products': 1,
 'Working hours (Sec.)': 6.88,
 'Number of operators': 1,
 'Cycle Time (Sec.)': 6.88,
 'Job saturation (%)': 0.86,
 'Value Judgment Process': 'VA',
 'operations': [{'serial_number': 1,
   'description': 'Take 1 PCS product in each hand and put it on the test machine support',
   'manual_sec': 2.01,
   'machine_sec': 0.0,
   'value_judgment_unit': 'NVA',
   'action_item_classification': 'Transport',
   'current_action_level': 'Automation'},
  {'serial_number': 2,
   'description': 'Press the switch with both hands to test, and wait for the test machine to display PASS',
   'manual_sec': 2.63,
   'machine_sec': 0.0,
   'value_judgment_unit': 'VA',
   'action_item_classification': 'Operation',
   'current_action_level': 'Semi automation'},
  {'serial_number': 3,
   'description': 'Put the good products into the production line',
   'manual_sec': 2.24,
   'machine_sec

In [189]:
# Saving the data in JSON format

import json
import numpy as np

def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert NumPy arrays to lists
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(elem) for elem in obj]
    return obj

# Assuming 'documents' is your data structure
serializable_documents = convert_to_serializable(documents)

with open("structured_data_no_metrics.json", "w") as f:
    json.dump(serializable_documents, f, indent=2)

In [2]:
import json
with open("structured_data_no_metrics.json", "r") as f:
    loaded_documents = json.load(f)

In [3]:
print(len(loaded_documents))


9


In [ ]:
texts = []
doc_ids = []
metadatas = []
for i, doc in enumerate(loaded_documents):
    # Combine fields into a single text string
    workstation_text = ", ".join([
        f"Workstation Number: {doc['Workstation number']}, "
        f"Workstation Name: {doc['Workstation name']}, "
        f"Number of Products: {doc['Number of products']}, "
        f"Working hours (Sec.): {doc['Working hours (Sec.)']}, "
        f"Number of Operators: {doc['Number of operators']}, "
        f"Cycle Time (Sec.): {doc['Cycle Time (Sec.)']}, "
        f"Job saturation (%): {doc['Job saturation (%)']}, "
        f"Value Judgment Process: {doc['Value Judgment Process']}, "])
    
    ops_text = ", ".join([
        f"Serial Number: {op['serial_number']}, " 
        f"Description: {op['description']}, " 
        f"Manual (sec): {op['manual_sec']}sec, " 
        f"Machine (sec): {op['machine_sec']}sec, " 
        f"Value Judgment Unit: {op['value_judgment_unit']}, " 
        f"Action Item Classification: {op['action_item_classification']}, " 
        f"Current Action Level: {op['current_action_level']}"
        for op in doc["operations"]])
    text = f"Workstation: {workstation_text}\nOperations: {ops_text}\nImprovements: {doc['improvement items']['goals'] if doc['improvement items']['goals'] != '/' else 'No improvements listed'}"

    texts.append(text)
    doc_ids.append(str(i))
    metadatas.append({
        "product_number": doc["product_number"],
        "workstation_number": str(doc["Workstation number"]),
        "workstation_name": doc["Workstation name"]
    })

In [5]:
print(texts[0])

Workstation: Workstation Number: 8, Workstation Name: 0/S Test, Number of Products: 1, Working hours (Sec.): 6.88, Number of Operators: 1, Cycle Time (Sec.): 6.88, Job saturation (%): 0.86, Value Judgment Process: VA, 
Operations: Serial Number: 1, Description: Take 1 PCS product in each hand and put it on the test machine support, Manual (sec): 2.01sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Transport, Current Action Level: Automation, Serial Number: 2, Description: Press the switch with both hands to test, and wait for the test machine to display PASS, Manual (sec): 2.63sec, Machine (sec): 0.0sec, Value Judgment Unit: VA, Action Item Classification: Operation, Current Action Level: Semi automation, Serial Number: 3, Description: Put the good products into the production line, Manual (sec): 2.24sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Transport, Current Action Level: Automation
Improvements: No improvements

In [6]:
import textwrap
print(len(doc_ids))
print(len(texts))
print(len(metadatas))

9
9
9


In [7]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")
all_embeddings = embed_model.embed_documents(texts)

In [8]:
import textwrap
print(len(doc_ids))
print(len(metadatas))
print(len(texts))

print(len(all_embeddings), len(all_embeddings[0]))


9
9
9
9 1536


In [44]:
print(texts[-1])

Workstation: Workstation Number: 17, Workstation Name: Full inspection & TR packaging, Number of Products: 1, Working hours (Sec.): 5.43, Number of Operators: 2, Cycle Time (Sec.): 2.715, Job saturation (%): 0.339375, Value Judgment Process: VA, 
Operations: Serial Number: 1, Description: Use both hands to take a group of products and place them on the packaging machine at the same time, Manual (sec): 2.03sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Transport, Current Action Level: Semi automation, Serial Number: 2, Description: Appearance inspection, Manual (sec): 2.06sec, Machine (sec): 0.0sec, Value Judgment Unit: VA, Action Item Classification: Inspect, Current Action Level: Manual, Serial Number: 3, Description: Use both hands to place the product in the packaging machine tray, Manual (sec): 1.34sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Operation, Current Action Level: Advanced automation
Improvements: No

In [15]:
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [9]:
import chromadb
# Initialize Chroma
## Create a collection for data injection

# Initialize Chroma
collection_name = "improvement_suggestions"
chroma_client = chromadb.PersistentClient(path="./chroma_db_6")

try:
    collection = chroma_client.create_collection(name=collection_name)
    logger.info(f"Created new collection: {collection_name}")
except Exception as e:
    if "already exists" in str(e).lower():
        collection = chroma_client.get_collection(name=collection_name)
        logger.info(f"Collection '{collection_name}' already exists. Using existing collection.")
    else:
        logger.error(f"Failed to create or access collection: {str(e)}")
        raise

NameError: name 'logger' is not defined

In [10]:
initial_count = collection.count()
print(f'Initial count#: {initial_count}')

Initial count#: 0


In [22]:
# chroma_client.delete_collection(name=collection_name)
# collection.count()

In [11]:
collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)
final_count = collection.count()
print(f'Final count#: {final_count}')

Final count#: 9


In [234]:
# Convert the text back to the JSON format to make the LLM life easy

def structure_document(text: str) -> str:
    """Convert comma-separated document to JSON."""
    fields = {}
    pairs = text.split(", ")
    for pair in pairs:
        if ":" in pair:
            key, value = pair.split(": ", 1)
            fields[key.strip()] = value.strip()
    return json.dumps(fields)

# During ingestion
structured_texts = [structure_document(text) for text in texts]

In [243]:
collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)

In [298]:
index = 8
print(doc_ids[index])
texts[index]

8


'Workstation Number: 17, Workstation Name: Full inspection & TR packaging, Number of Products: 1, Working hours (Sec.): 5.43, Number of Operators: 2, Cycle Time (Sec.): 2.715, Job saturation (%): 0.339375, Value Judgment Process: VA, . Serial Number: 1, Description: Use both hands to take a group of products and place them on the packaging machine at the same time, Manual (sec): 2.03sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Transport, Current Action Level: Semi automation, Serial Number: 2, Description: Appearance inspection, Manual (sec): 2.06sec, Machine (sec): 0.0sec, Value Judgment Unit: VA, Action Item Classification: Inspect, Current Action Level: Manual, Serial Number: 3, Description: Use both hands to place the product in the packaging machine tray, Manual (sec): 1.34sec, Machine (sec): 0.0sec, Value Judgment Unit: NVA, Action Item Classification: Operation, Current Action Level: Advanced automation. No improvements listed.'

RAG

In [12]:
## GROK Suggestions
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
def validate_collection_inputs(ids, embeddings, documents, metadatas):
    if not all([ids, embeddings, documents]):  # Check for empty inputs
        raise ValueError("IDs, embeddings, and documents must not be empty")
    if len(ids) != len(embeddings) or len(ids) != len(documents):
        raise ValueError("IDs, embeddings, and documents must have the same length")
    if metadatas and len(metadatas) != len(ids):
        raise ValueError("Metadatas length must match IDs length if provided")

try:
    validate_collection_inputs(doc_ids, all_embeddings, texts, metadatas)
    collection.add(ids=doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)
    logger.info("Successfully added data to collection")
except Exception as e:
    logger.error(f"Failed to add data to collection: {str(e)}")
    raise

2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 0
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 1
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 2
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 3
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 4
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 5
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 6
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 7
2025-04-21 10:27:02,222 - WARNING - Add of existing embedding ID: 8
2025-04-21 10:27:02,222 - WARNING - Insert of existing embedding ID: 0
2025-04-21 10:27:02,222 - WARNING - Insert of existing embedding ID: 1
2025-04-21 10:27:02,222 - WARNING - Insert of existing embedding ID: 2
2025-04-21 10:27:02,222 - WARNING - Insert of existing embedding ID: 3
2025-04-21 10:27:02,234 - WARNING - Insert of existing embedding ID: 4
2025-04-21 10:27:02,235 - WARNING

In [13]:

# import chromadb
# # Initialize Chroma
# ## Create a collection for data injection
# collection_name = "improvement_suggestions"
# chroma_client = chromadb.PersistentClient(path="./chroma_db_5")

# try:
#     collection = chroma_client.create_collection(name=collection_name)
# except Exception as e:
#     if "already exists" in str(e):
#         # If the collection already exists, use the existing one
#         collection = chroma_client.get_collection(name=collection_name)
#         print(f"Collection '{collection_name}' already exists. Using the existing collection.")
#     else:
#         # Handle other exceptions
#         raise e


# collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)





from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import textwrap
llm = ChatOpenAI(temperature = 0, model = "gpt-4o",)


template ="""You will recieve the user's question along with the search results of that question \
over a database. Give the user the proper answer.
User's question: {query_texts} 
Search results: {results}
When a question is involved regarding an operation, please fully mention that in your answer.
"""
prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

In [ ]:
The workstations of this production line are:
1. Solder ring assembly & press-in (Workstation Number: 10)
2. Long and short PIN test (Workstation Number: 15)
3. Full inspection & TR packaging (Workstation Number: 17)
4. Coplanarity test 1 (Workstation Number: 12)
5. 0/S Test (Workstation Number: 8)
6. Internal GAP inspection (CCD inspection) (Workstation Number: 13)
7. Hi-pottest&Position detection (Workstation Number: 9)
8. Coplanarity test (retest) (Workstation Number: 14)

In [14]:
query_texts = "Please analyze the workstation 'Coplanarity test (retest)' and provide improvement suggestions upon ECRS"
query_embeddings = embed_model.embed_query(query_texts)
# Load the chromadb collection for vector search
vectordb = chroma_client.get_collection(name="improvement_suggestions")
# Retrieve relevant chunk
results = vectordb.query(
    query_embeddings = query_embeddings,
    # n_results=1 #top_k
)
response = chain.invoke({'query_texts': query_texts, 'results': results})
print(response)

2025-04-21 10:27:13,967 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-21 10:27:13,967 - WARNING - Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9
2025-04-21 10:27:26,301 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The workstation "Coplanarity test (retest)" is identified as Workstation Number 14. Here is an analysis based on the ECRS (Eliminate, Combine, Rearrange, Simplify) principles, along with improvement suggestions:

### Current Operations:
1. **Operation 1:**
   - **Description:** Take 1 PCS product with your left hand and place it on the AOI inspection machine, and press the button with your right hand to test it.
   - **Manual Time:** 3.21 seconds
   - **Value Judgment Unit:** NVA (Non-Value Added)
   - **Action Item Classification:** Transport
   - **Current Action Level:** Automation

2. **Operation 2:**
   - **Description:** Take the product with your left hand, pick up the tool with your right hand, make adjustments according to the defects shown on the display, and then put the product on the AOI for testing.
   - **Manual Time:** 5.24 seconds
   - **Value Judgment Unit:** VA (Value Added)
   - **Action Item Classification:** Operation
   - **Current Action Level:** Semi-automation

In [15]:
results['distances']

[[0.9521121969299092,
  0.9976683482169735,
  1.0560022252413304,
  1.0590029763851334,
  1.0741560611091794,
  1.1206011370919342,
  1.1276897716395842,
  1.1339480257947283,
  1.1751301530005756]]

In [377]:
for key, value in results.items():
    print(key, value)

ids [['op_4_2', 'op_6_2', 'op_0_0', 'ws_0', 'op_0_2', 'op_4_1', 'op_0_1', 'op_6_1', 'ws_6', 'op_4_0']]
embeddings None
documents [['Workstation Number: 12; Workstation Name: Coplanarity test 1; Serial Number: 3; Description: Repeat the above steps until all the displays are good.; Manual (sec): 23.12sec; Machine (sec): 0.0sec; Value Judgment Unit: NVA; Action Item Classification: Operation; Current Action Level: Semi automation', 'Workstation Number: 14; Workstation Name: Coplanarity test (retest); Serial Number: 3; Description: Repeat the above steps until all the displays are good.; Manual (sec): 4.37sec; Machine (sec): 0.0sec; Value Judgment Unit: NVA; Action Item Classification: Operation; Current Action Level: Semi automation', 'Workstation Number: 8; Workstation Name: 0/S Test; Serial Number: 1; Description: Take 1 PCS product in each hand and put it on the test machine support; Manual (sec): 2.01sec; Machine (sec): 0.0sec; Value Judgment Unit: NVA; Action Item Classification: Tr

In [231]:
len(structured_texts[0])
len(structured_texts[1])

556

In [232]:
structured_texts[1]

'{"Workstation Number": "9", "Workstation Name": "Hi-pottest&Position detection", "Number of Products": "1", "Working hours (Sec.)": "7.2", "Number of Operators": "1", "Cycle Time (Sec.)": "7.2", "Job saturation (%)": "0.9", "Value Judgment Process": "VA", ". Serial Number": "1", "Description": "Put the good products into the production line", "Manual (sec)": "0.82sec", "Machine (sec)": "0.0sec", "Value Judgment Unit": "NVA", "Action Item Classification": "Transport", "Current Action Level": "Automation. No improvements listed.", "Serial Number": "4"}'

In [2]:
word = 'bangladesh'
for i in word:
    print(i)

b
a
n
g
l
a
d
e
s
h
